# Guide to: insert missing residues

> this is a notebook to guide the modeller template procedure, the actual computation is a remix of this
>
> WARNING: before running this script, make sure to setup MODELLER. I suggest to use a conda environment for this. Moreover, download a input structure to complete, in the following, the example is given by the `1QG8` structure.
>
> Packages required: Biopython, MODELLER

Adapted from: https://salilab.org/modeller/wiki/Missing_residues

Often, you will encounter files in the PDB which have missing residues. Special care must be taken in this case, as MODELLER only reads the ATOM and HETATM records, not the SEQRES records, and so will not handle missing residues automatically. **Unfortunately PDB is not reliable enough to be able to automatically rely on SEQRES**. Therefore, since I want to automate this process using the SEQRES entry, I need to add extra controls such as:
- reading REMARK 465 for missing residues

One example is PDB code 1qg8, which is missing residues 134-136 and 218-231 (see the REMARK 465 lines in the PDB file). We can use Modeller to 'fill in' these missing residues by treating the original structure (with missing residues) as a template, and building a comparative model using the full sequence extracted from the SEQRES.

In [1]:
import modeller

code = '1QG8'
input = "./input/"
output = "./output/"

# create modeller environment; starting point for all modeller work
env = modeller.Environ()

# create model and sequence file for the original structure
# the one with missing residues 
model = modeller.Model(env, file=input + code)

# Alignment object used to align sequence with missing residues
# with the one complete
alignment = modeller.Alignment(env)
alignment.append_model(model, align_codes=code)
alignment.write(file=output + code + '.seq')


                         MODELLER 10.4, 2022/10/28, r12463

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2022 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Linux lammps-salicari 5.4.0-144-generic x86_64
Date and time of com

Now we perform the alignment: from the PDB REMARKs or SEQRES records, we know the missing residues. Hence, we can make an alignment between the original 1qg8 structure (as the template), with gap characters corresponding to the missing residues, and the full sequence.

First, we read the full sequence from the SEQRES using the Biopython library and wrote it as fasta file.
The `record.id` string is used to characterise the file as `PDBID:CHAINID` (see [biopython implementation](https://github.com/biopython/biopython/blob/fadc43625b3bbb47364e3b80106034079786c25c/Bio/SeqIO/PdbIO.py#L177)).
Hence, in the code below, there is a record for each polymer entry (to check!)

Resources:
- https://biopython.org/wiki/SeqIO
- https://biopython.org/wiki/SeqRecord
- https://biopython.org/docs/1.75/api/Bio.SeqIO.PdbIO.html#Bio.SeqIO.PdbIO.PdbSeqresIterator

In [2]:
from Bio import SeqIO
for record in SeqIO.parse(input+f"{code}.pdb", "pdb-seqres"):
    print("Record id %s, chain %s" % (record.id, record.annotations["chain"]))
    record.id = record.id + "_complete" # suffix used to distinguish it from the one with missing residues
    SeqIO.write(record, output+f"{record.id}.fasta", "fasta")

Record id 1QG8:A, chain A


Now, we load the complete sequence into the alignment object and perform the alignment with modeller.
All results are saved in the `.ali` file

Resources:
- https://salilab.org/modeller/10.4/manual/node296.html
- https://salilab.org/modeller/10.4/manual/node318.html#CMD:Alignment.salign

In [3]:
# load the complete sequence into the previous alignment object
alignment.append(file=output+f"{record.id}.fasta", alignment_format='fasta')
alignment.salign() # perform the actual alignment
alignment.write(file=output+f'{record.id}.ali', alignment_format='PIR')
del alignment # to avoid running multiple times this cell


SALIGN_____> adding the next group to the alignment; iteration    1


Now we prepare the `AutoModel` object that will perform the modelling.
In particular, we define a custom class that inherits AutoModel in order to model only the section we are interested in. This speeds up the computation by roughly 3 times!

Note that the selection is used to keep fixed all those residues which are not modeled

Resources:
- https://salilab.org/modeller/10.4/manual/node47.html
- https://salilab.org/modeller/manual/node69.html
- https://salilab.org/modeller/manual/node23.html

In [4]:
from modeller.automodel import AutoModel

modeller.log.verbose()
env = modeller.Environ()

env.io.atom_files_directory = ['./input/']

class ModelSegment(AutoModel):
    def select_atoms(self):
        return modeller.Selection(self.residue_range('134:A', '136:A'),
                         self.residue_range('218:A', '231:A'))

loopmodel = ModelSegment(
    env,
    alnfile=output+f'{record.id}.ali',
    knowns=code, # sequence to be completed
    sequence=f'{record.id}' # target (complete) sequence
)
# the following force to build only one model
loopmodel.starting_model    = 1
loopmodel.ending_model      = 1

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL10v4}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL10v4}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       788841     770.353     0.752

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       789369     770.868     0.753
openf___224_> Open           ${MODINSTALL10v4}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       837969     818.329     0.799
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL10v4}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       851269     831.317     0.812
openf___224_> Open           $

Trigger the actual run. The following are performed to build the model:
- homology search
- a combination of energy minimization and brief NVT simulations (see `.sch` file)

In [5]:
loopmodel.make()

# move the output file to the output directory
import shutil
import os
source = "./"
files_list = [file for file in os.listdir(source) if file.startswith(code)]
for files in files_list:
    shutil.move(files, output)

openf___224_> Open           ./output/1QG8:A_complete.ali

Dynamically allocated memory at   amaxalignment [B,KiB,MiB]:       863042     842.814     0.823

Dynamically allocated memory at   amaxalignment [B,KiB,MiB]:       864492     844.230     0.824

Dynamically allocated memory at   amaxalignment [B,KiB,MiB]:       867392     847.062     0.827

Dynamically allocated memory at   amaxalignment [B,KiB,MiB]:       873192     852.727     0.833

Dynamically allocated memory at   amaxalignment [B,KiB,MiB]:      1002400     978.906     0.956

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:      1003348     979.832     0.957

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:      1004364     980.824     0.958

Read the alignment from file       : ./output/1QG8:A_complete.ali

Total number of alignment positions:   255

  #  Code        #_Res #_Segm PDB_code    Name
-------------------------------------------------------------------------------
  1       1QG8     238  

As outputs, modeller dump various files that describes the minimization procedure as well as the final modelled structure.
A bried recap, also if the run was succesful. are stored in the `outputs` variable of the `AutoModel` object

Resources:
- https://salilab.org/modeller/10.4/manual/node31.html#SECTION:model-outputs
- https://salilab.org/modeller/manual/node267.html#CLASS:physvalues
- https://salilab.org/modeller/methenz/andras/node15.html

In [6]:
# check if run was successful
if loopmodel.outputs[0]["failure"] is None:
    print("Success!")
# save output file name and rename it
final_output_file = loopmodel.outputs[0]["name"]
shutil.copyfile(output+final_output_file, output+f"{final_output_file[:5]}_fixed.pdb")

Success!


'./output/1QG8A_fixed.pdb'

## Addendum

From the [FAQs](https://salilab.org/modeller/FAQ.html#14)

**How do I prevent “knots” in the final models?**

The best way to prevent knots is to start with a starting structure that is as close to the desired final model as possible. Other than that, the only solution at this point is to calculate independently many models and hope that in some runs there won't be knots. Knots usually occur when one or more neighboring long insertions (i.e., longer than 15 residues) are modeled from scratch. The reason is that an insertion is built from a randomized distorted structure that is located approximately between the two anchoring regions. Under such conditions, it is easy for the optimizer to “fall” into a knot and then not be able to recover from it. Sometimes knots result from an incorrect alignment, especially when more than one template is used. When the alignment is correct, knots are a result of optimization not being good enough. However, making optimization more thorough by increasing the CPU time would not be worth it on the average as knots occur relatively infrequently. The excluded volume restraints are already included in standard comparative modeling with the AutoModel class